In [ ]:
!pip install transformers accelerate datasets --quiet

import pandas as pd
import torch
from tqdm import tqdm
import gc
from transformers import AutoTokenizer, AutoModelForCausalLM


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.4 MB/s eta 0:00:00


In [ ]:
import os

# Paste your actual token inside the quotes
os.environ["HF_TOKEN"] = "get-your-token-from-huggingface"
hf_token = os.environ["HF_TOKEN"]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/blogdata.csv")

# Ensure the key columns exist
df["title"] = df.get("title", "")
df["content"] = df.get("content", "")


Mounted at /content/drive


In [ ]:
import os
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm

# Load Gemma 2B model and tokenizer (requires authentication)
model_id = "google/gemma-2b-it"
token = os.environ["HF_TOKEN"]  # Make sure HF_TOKEN is set in the environment

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=token,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
)

# Text generation function that removes the prompt from the returned output
def generate_text(prompt, max_tokens=512):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    # Remove prompt echo if present
    if decoded.startswith(prompt):
        return decoded[len(prompt):].strip()
    return decoded.strip()

# Rewrite function for either title or content
def rewrite_field(text, field_type="title"):
    if field_type == "title":
        prompt = f"""
You are an assistant that rewrites blog headlines.
Rewrite the following blog title to be clear, professional, and engaging.
Only output ONE rewritten title. Do not include explanations or formatting.

Title: {text}
"""
        return generate_text(prompt, max_tokens=40)

    elif field_type == "content":
        prompt = f"""
You are an assistant that rewrites blog articles.
Rewrite the following blog content to be clear, concise, and professionally written.
Output only the rewritten blog content with no commentary, explanation, or formatting.

Content: {text}
"""
        return generate_text(prompt, max_tokens=768)

# Load your CSV
# df = pd.read_csv("your_data.csv")

# Rewrite loop (batch of 5 rows at a time)
new_titles = []
new_contents = []

batch_size = 5
for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i+batch_size]

    for idx, row in batch.iterrows():
        original_title = str(row["title"]).strip()
        original_content = str(row["content"]).strip()

        rewritten_title = rewrite_field(original_title, field_type="title")
        rewritten_content = rewrite_field(original_content, field_type="content")

        new_titles.append(rewritten_title)
        new_contents.append(rewritten_content)

# Add results to dataframe
df["rewritten_title"] = new_titles
df["rewritten_content"] = new_contents


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
# Save only the rewritten title and content
output_df = df[["rewritten_title", "rewritten_content"]].copy()
output_df.columns = ["title", "content"]  # Rename columns to match expected output format

# Define output path and save CSV
output_path = "/content/drive/MyDrive/rewritten_blogs_clean.csv"
output_df.to_csv(output_path, index=False, encoding="utf-8")
print(f"✅ Done. Rewritten blog content saved to {output_path}")


✅ Done. Rewritten blog content saved to /content/drive/MyDrive/rewritten_blogs_clean.csv
